In [ ]:
pip install -U openai

In [1]:
import pandas as pd
import json
from dotenv import load_dotenv
import os
import openai
#from openai import OpenAI
from openai import AzureOpenAI
import mysql.connector
from sqlalchemy import create_engine
import mysql
import pymysql
import re

## SYSTEM MASSAGE (PROMPT FOR OPENAI)

In [40]:
OPENAI_PROMPT = """You are an AI assistant that is able to convert natural language into a properly formatted SQL query.

The database you will be querying is called "sakila". Here is the schema of the database:
{schema}

You must always output your answer in JSON format with the following key-value pairs (without the "json" text at the beginning of the output):
{{- "chain of thought": the reasoning or story behind the generated query
- "query": the SQL query that you generated
- "error": an error message if the query is invalid, or null if the query is valid}}

here is the correct output, dont use anything other than this following output example:

{{"chain_of_thought": "The user is asking for the description of all tables in the sakila database. To achieve this, we would typically use the 'DESCRIBE' or 'SHOW COLUMNS FROM' command in SQL for each table. However, since the user is asking for the description of all tables, we would use the 'SHOW TABLES' command to get the list of tables and then describe each one. But as an AI, I can only generate a single SQL query at a time, so I will provide the SQL command to list all the tables, which is the first step in the process."
"query": "SELECT title, length FROM film ORDER BY length DESC LIMIT 1;",
"error": null}}

"""

## GET OPENAI

In [41]:
OPENAI_API_TYPE = "xxx" 
OPENAI_API_VERSION = "xxx" 
OPENAI_API_BASE = "xxx" 
OPENAI_API_KEY = "xxx" 
#GPT3.5 --> Deployment Name: gpt-35-turbo 
#Embeding Engine —> Deployment Name: text-embedding-ada-002

def get_completion_from_messages(system_message, user_message, model="gpt-35-turbo", temperature=0, max_tokens=500) -> str:
  client = AzureOpenAI(
    azure_endpoint = OPENAI_API_BASE, 
    api_key=OPENAI_API_KEY,  
    api_version=OPENAI_API_VERSION
  )

  response = client.chat.completions.create(
      model="gpt-35-turbo",
          messages = [
          {'role': 'system', 'content': system_message},
          {'role': 'user', 'content': f"{user_message}"}
      ]
  )

  return response.choices[0].message.content

## CONNECTING TO DATABASE

In [42]:
cnx = mysql.connector.connect(
  host="xxx",
  user="xxx",
  password="xxx",
  database = 'xxx'
)

In [43]:
# take database schema
cursor=cnx.cursor()
query=("SELECT  TABLE_NAME, COLUMN_NAME, DATA_TYPE, COLUMN_KEY FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA='sakila';")
cursor.execute(query)
data = []
for row in cursor:
  data.append(row)

df = pd.DataFrame(data, columns=cursor.column_names)
df

,TABLE_NAME,COLUMN_NAME,DATA_TYPE,COLUMN_KEY
0,actor,actor_id,smallint,PRI
1,actor,first_name,varchar,
2,actor,last_name,varchar,MUL
3,actor,last_update,timestamp,
4,actor_info,actor_id,smallint,
...,...,...,...,...
127,staff_list,SID,tinyint,
128,store,store_id,tinyint,PRI
129,store,manager_staff_id,tinyint,UNI
130,store,address_id,smallint,MUL


## MAIN APP

In [54]:
# pass the database schema into prompt
formatted_system_message = OPENAI_PROMPT.format(schema=df)

# text input
user_message = "show me all city that starts with A"

# use openai to generate query out of it
response = get_completion_from_messages(formatted_system_message, user_message)
json_response = json.loads(response)
json_response['query']

"SELECT city FROM city WHERE city LIKE 'A%';"

## EXECUTING QUERY IN DATABASE

In [55]:
# pass the generated query into database connector
cursor=cnx.cursor()
querys=(json_response['query'])
cursor.execute(querys)
data = []
for row in cursor:
  data.append(row)

df = pd.DataFrame(data, columns=cursor.column_names)
df

,city
0,A Coruña (La Coruña)
1,Abha
2,Abu Dhabi
3,Acuña
4,Adana
5,Addis Abeba
6,Aden
7,Adoni
8,Ahmadnagar
9,Akishima
